# Build Your Own SFINCS Hydrodynamic Model

## Overview

This interactive notebook allows you to build a custom **SFINCS hydrodynamic model** for any location in East Africa. Using an interactive map, you can select your model region, configure model parameters, and generate a complete hydrodynamic model ready for simulation.


### What You'll Do

1. **Select your model region** using an interactive map
2. **Configure model parameters** including grid resolution and forcing data
3. **Build your model** automatically using HydroMT-SFINCS
4. **Visualize the results** to verify your model setup
---

## Prerequisites

Before starting, ensure you have:
- Completed the SFINCS training notebook (1_Run_DireDawa_SFINCS_model.ipynb)
- Selected the **"HydroMT SFINCS (pixi)"** kernel (top-right corner)
- Prepared your data catalogs and configuration files

---


In [ ]:
# Standard library imports
import os
from pathlib import Path
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
from shapely.geometry import Point, box
import matplotlib.pyplot as plt
import folium


## Part 1: Interactive Map for Region Selection

### Create Interactive Map

Use this interactive map to:
- **Visualize** reference layers (coastal areas, rivers, elevation)
- **Click** to define your model region bounding box
- **View** the coordinates of your selected area

The map will display your click coordinates in latitude and longitude.

In [ ]:
# define a point for a city
# nairobi
lat = -1.286389
lon = 36.817223

selected_coords = {'lat': None, 'lon': None, 'bbox': None}

import folium
from folium import plugins 

def create_interactive_map(
    center=[9.5, 41.0], 
    zoom=7, 
    geojson_files=None):
    """
    Create an interactive Folium map with click functionality for SFINCS model setup.
    
    Parameters:
    -----------
    center : list
        [latitude, longitude] for map center (default: Dire Dawa region)
    zoom : int
        Initial zoom level
    geojson_files : dict
        Dictionary of {layer_name: file_path} for GeoJSON layers
    
    Returns:
    --------
    folium.Map
        Interactive map object
    """
    # Create base map
    m = folium.Map(
        location=center,
        zoom_start=zoom,
        tiles='OpenStreetMap',
        control_scale=True
    )
    
 # Add alternative base layers
    folium.TileLayer(
        tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr='Esri',
        name='Satellite',
        overlay=False,
        control=True
    ).add_to(m)
    
    # Add satellite imagery with labels overlay
    folium.TileLayer(
        tiles='https://server.arcgisonline.com/ArcGIS/rest/services/Reference/World_Boundaries_and_Places/MapServer/tile/{z}/{y}/{x}',
        attr='Esri',
        name='Labels',
        overlay=True,
        control=True,
        show=True
    ).add_to(m)
    
    
    # Add GeoJSON layers if provided
    if geojson_files:
        colors = ['blue', 'green', 'red', 'purple', 'orange']
        for idx, (layer_name, file_path) in enumerate(geojson_files.items()):
            if Path(file_path).exists():
                try:
                    color = colors[idx % len(colors)]
                    folium.GeoJson(
                        str(file_path),
                        name=layer_name,
                        style_function=lambda x, c=color: {
                            'fillColor': c,
                            'color': c,
                            'weight': 2,
                            'fillOpacity': 0.3
                        }
                    ).add_to(m)
                except Exception as e:
                    print(f"Warning: Could not load {layer_name}: {e}")
    
    # Add click functionality for coordinate capture
    m.add_child(folium.LatLngPopup())
    
    # Add drawing tools for bounding box
    draw = plugins.Draw(
        export=False,
        draw_options={
            'polyline': False,
            'rectangle': True,
            'polygon': True,  # Enable polygon drawing
            'circle': False,
            'circlemarker': False,
            'marker': True
        }
    )
    draw.add_to(m)
    
    # Add measure control
    plugins.MeasureControl(position='topleft', primary_length_unit='kilometers').add_to(m)
    
    # Add layer control
    folium.LayerControl().add_to(m)
    
    # Add custom HTML for instructions
    legend_html = '''
    <div style="position: fixed; 
                top: 10px; right: 10px; width: 280px; height: auto; 
                background-color: white; z-index:9999; font-size:14px;
                border:2px solid grey; border-radius: 5px; padding: 10px">
    <h4 style="margin-top: 0;">SFINCS Model Setup</h4>
    <p><b>1.</b> Use the rectangle tool (◼) to define your model domain</p>
    <p><b>2.</b> Click on the map to see coordinates</p>
    <p><b>3.</b> Copy the bounding box coordinates</p>
    <p><b>4.</b> Enter them in the widget below</p>
    </div>
    '''
    m.get_root().html.add_child(folium.Element(legend_html))
    
    return m

# Create the map
print("Creating interactive map...")
map_widget = create_interactive_map(
    center=[9.5, 41.0],  # Default center (Dire Dawa region)
    zoom=7,
)

# Display the map
display(map_widget)

In [ ]:
# create as geodataframe of
from shapely.geometry import shape
import geopandas as gpd

shp = {"type":"Feature","properties":{},"geometry":{"type":"Polygon","coordinates":[[[36.695023,-1.263325],[36.666183,-1.321675],[36.786346,-1.386202],[36.822052,-1.38277],[36.852951,-1.320302],[36.890717,-1.35188],[36.962128,-1.304514],[36.920242,-1.23312],[36.853638,-1.202915],[36.825485,-1.218017],[36.78154,-1.215958],[36.748581,-1.237239],[36.695023,-1.263325]]]}}
polygon = shape(shp['geometry'])

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame({'geometry': [polygon]}, crs="EPSG:4326")
gdf.plot()


In [ ]:
# save as region.geojson
gdf.to_file("region.geojson", driver='GeoJSON')

## Part 2: Build your own SFINCS model

In [ ]:
# To check the version of hydromt and the hydromt_sfincs plugin, run the following command in a terminal:
!/opt/miniforge3/envs/hydromt_sfincs/bin/hydromt --models

Below we will set the region to "region.geojson" using the `--region` flag. 

In [ ]:
! /opt/miniforge3/envs/hydromt_sfincs/bin/hydromt build sfincs my_own_sfincs_model --region "{'geom': 'region.geojson'}" -i ../model_build_files/sfincs_build_my_own_model.yml --force-overwrite -v -d ../model_build_files/hydromt_east_africa_sfincs.yml